In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
#import the weather files for top 20 origin airports of 2018 obtained from:
# https://mesonet.agron.iastate.edu/ASOS/
path = './Data/Weather/'
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f, parse_dates=['valid']) for f in all_files)
concatenated_df = pd.concat(df_from_each_file)

/Users/dmitriykats/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  import sys
/Users/dmitriykats/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (15,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  import sys
/Users/dmitriykats/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  import sys
/Users/dmitriykats/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [3]:
weather_df = concatenated_df

In [4]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 100)
pd.set_option('display.max_info_columns', 200)

In [5]:
weather_df.columns

Index(['alti', 'drct', 'dwpf', 'feel', 'gust', 'ice_accretion_1hr', 'ice_accretion_3hr',
       'ice_accretion_6hr', 'jfstation', 'lcstation', 'metar', 'mslp', 'p01i', 'peak_wind_drct',
       'peak_wind_gust', 'peak_wind_time', 'relh', 'sknt', 'skyc1', 'skyc2', 'skyc3', 'skyc4',
       'skyl1', 'skyl2', 'skyl3', 'skyl4', 'station', 'tmpf', 'valid', 'vsby', 'wxcodes'],
      dtype='object')

In [7]:
weather_df = weather_df[['station','valid', 'tmpf','vsby','gust','sknt','p01i',
                         'ice_accretion_3hr','ice_accretion_6hr',]]

In [8]:
weather_df.columns

Index(['station', 'valid', 'tmpf', 'vsby', 'gust', 'sknt', 'p01i', 'ice_accretion_3hr',
       'ice_accretion_6hr'],
      dtype='object')

In [22]:
weather_df = weather_df.fillna(0)

In [21]:
weather_df['p01i'] = pd.to_numeric(weather_df.p01i, errors='coerce')

In [9]:
weather_df['valid_new'] = weather_df.valid.apply(lambda x: x.replace(minute=0))

/Users/dmitriykats/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
df1 = pd.DataFrame(weather_df.groupby(['station','valid_new'])['tmpf','vsby'].min())

In [23]:
df2 = pd.DataFrame(weather_df.groupby(['station','valid_new'])['p01i'].max())

In [24]:
df3 = pd.DataFrame(weather_df.groupby(['station','valid_new'])['gust','sknt'].max())

In [25]:
df_test = pd.merge(df1, df2, how='left', left_index=True, right_index=True)

In [26]:
df_test = df_test.merge(df3, how='left', left_index=True, right_index=True)

In [27]:
df_test.head()

tmpf  vsby  p01i  gust  sknt
station valid_new                                        
ATL     2018-01-01 00:00:00  26.1  10.0   0.0  22.0  17.0
        2018-01-01 01:00:00  25.0  10.0   0.0  19.0  16.0
        2018-01-01 02:00:00  23.0  10.0   0.0   0.0  16.0
        2018-01-01 03:00:00  21.9  10.0   0.0  20.0  15.0
        2018-01-01 04:00:00  21.0  10.0   0.0   0.0  17.0

In [28]:
df_test.reset_index(inplace=True) 

In [29]:
df_test.head()

,station,valid_new,tmpf,vsby,p01i,gust,sknt
0,ATL,2018-01-01 00:00:00,26.1,10.0,0.0,22.0,17.0
1,ATL,2018-01-01 01:00:00,25.0,10.0,0.0,19.0,16.0
2,ATL,2018-01-01 02:00:00,23.0,10.0,0.0,0.0,16.0
3,ATL,2018-01-01 03:00:00,21.9,10.0,0.0,20.0,15.0
4,ATL,2018-01-01 04:00:00,21.0,10.0,0.0,0.0,17.0


In [30]:
weather_df_new = df_test.copy().fillna(0)

In [31]:
weather_df_new.head()

,station,valid_new,tmpf,vsby,p01i,gust,sknt
0,ATL,2018-01-01 00:00:00,26.1,10.0,0.0,22.0,17.0
1,ATL,2018-01-01 01:00:00,25.0,10.0,0.0,19.0,16.0
2,ATL,2018-01-01 02:00:00,23.0,10.0,0.0,0.0,16.0
3,ATL,2018-01-01 03:00:00,21.9,10.0,0.0,20.0,15.0
4,ATL,2018-01-01 04:00:00,21.0,10.0,0.0,0.0,17.0


In [32]:
#export raw, concoctanated weather file for 2017 top 20 airports
weather_df_new.to_csv('./Data/Weather/2018WeatherALL.csv', index = False, encoding = 'utf-8')

In [33]:
#import the 2018 Data and clean it
df = pd.read_csv('./Data/2018ALL.csv', parse_dates=['FlightDate'])


df = df.dropna(subset=['Tail_Number','DepTime','DepDelay','DepartureDelayGroups','TaxiOut',
                      'WheelsOff','WheelsOn','TaxiIn','ArrTime','ArrDelay','CRSElapsedTime',
                      'ActualElapsedTime','AirTime'])

df = df.fillna(0)

In [34]:
#Let's remove the features that will not be available for predicting flights
df = df[['Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline',
       'Origin', 'OriginState', 'DestState', 'Dest', 'CRSDepTime', 'CRSArrTime', 'CRSElapsedTime',
       'ArrDelay', 'Distance', 'DistanceGroup',]]
df = df.reindex()

In [35]:
#since the DEP_TIME is presented as a float we need to convert it to a Timestamp
def convert_to_time (datatype):
    '''This method will take a float as input and returns a datetime.time object'''
    to_string = str(int(datatype))
    #check length of string and add leading zeros if lenght is less than 4
    while len(to_string) < 4:
        to_string = '0' + to_string
    #check if hour starts with 24 then change to 00
    if to_string[:2] == '24':
        to_string = '00' + to_string[2:]
    #obj = time.strptime(to_string, '%H%M')
    
    to_string = to_string[:2] + ':' + to_string[2:] 
    datetime = pd.to_datetime(to_string)
    datetime = pd.to_datetime(to_string, format='%H:%M')
    time = datetime.time()
    return time
    
    #datetime = time.strftime("%H:%M", obj)
    #return datetime
    
#We now need to create a new Timestamp column incorporating the information from FL_DATE and DEP_TIME
def combine_datetime (dataframe):
    '''This function will extract needed data from two columns and combine into a single Timestamp object'''
    year = dataframe[0].year
    month = dataframe[0].month
    day = dataframe[0].day
    hour = dataframe[1].hour
    minute = dataframe[1].minute
    second = dataframe[1].second
    
    datetime = pd.Timestamp(year, month, day, hour, minute, second)
    return datetime

In [36]:
combined = df[['FlightDate','CRSDepTime']]

In [37]:
#convert DEP_TIME to timestamp
combined['CRSDepTime'] = df.CRSDepTime.apply(convert_to_time)

/Users/dmitriykats/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [38]:
#apply the combine_datetime function to optimized_df and create a new column 'FL_TIME'
df['FL_TIME'] = combined.apply(combine_datetime, axis=1)

In [39]:
df['FL_TIME_NEW'] = df.FL_TIME.apply(lambda x: x.replace(minute=0))

In [40]:
df.head()

,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,Origin,OriginState,DestState,Dest,CRSDepTime,CRSArrTime,CRSElapsedTime,ArrDelay,Distance,DistanceGroup,FL_TIME,FL_TIME_NEW
0,4,10,14,7,2018-10-14,AA,N925UY,1674,LAX,CA,NC,CLT,1123,1910,287.0,-16.0,2125.0,9,2018-10-14 11:23:00,2018-10-14 11:00:00
1,4,10,15,1,2018-10-15,AA,N583UW,1674,LAX,CA,NC,CLT,1123,1910,287.0,-21.0,2125.0,9,2018-10-15 11:23:00,2018-10-15 11:00:00
2,4,10,16,2,2018-10-16,AA,N581UW,1674,LAX,CA,NC,CLT,1123,1910,287.0,-14.0,2125.0,9,2018-10-16 11:23:00,2018-10-16 11:00:00
3,4,10,17,3,2018-10-17,AA,N551UW,1674,LAX,CA,NC,CLT,1123,1910,287.0,-8.0,2125.0,9,2018-10-17 11:23:00,2018-10-17 11:00:00
4,4,10,18,4,2018-10-18,AA,N181UW,1674,LAX,CA,NC,CLT,1123,1910,287.0,-4.0,2125.0,9,2018-10-18 11:23:00,2018-10-18 11:00:00


In [43]:
weather_df_new1 = weather_df_new.copy()

In [44]:
#remove rows will null values which is approximately 1.5%
null_columns = weather_df_new1.columns[weather_df_new1.isnull().any()]
weather_df_new1[null_columns].isnull().sum()

Series([], dtype: float64)

In [45]:
weather_df_new.head()

,station,valid_new,tmpf,vsby,p01i,gust,sknt
0,ATL,2018-01-01 00:00:00,26.1,10.0,0.0,22.0,17.0
1,ATL,2018-01-01 01:00:00,25.0,10.0,0.0,19.0,16.0
2,ATL,2018-01-01 02:00:00,23.0,10.0,0.0,0.0,16.0
3,ATL,2018-01-01 03:00:00,21.9,10.0,0.0,20.0,15.0
4,ATL,2018-01-01 04:00:00,21.0,10.0,0.0,0.0,17.0


In [46]:
#combine the station name and datetime for easier / faster merging with flight data
weather_df_new1 = pd.DataFrame()
weather_df_new1['combined'] = weather_df_new['station'] + weather_df_new['valid_new'].astype(str)
weather_df_new1['precip'] = weather_df_new['p01i']
weather_df_new1['vis'] = weather_df_new['vsby']
weather_df_new1['temp'] = weather_df_new['tmpf']
weather_df_new1['wind'] = weather_df_new['sknt']

In [47]:
weather_df_new1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157154 entries, 0 to 157153
Data columns (total 5 columns):
combined    157154 non-null object
precip      157154 non-null float64
vis         157154 non-null float64
temp        157154 non-null float64
wind        157154 non-null float64
dtypes: float64(4), object(1)
memory usage: 6.0+ MB


In [48]:
weather_df_new1.head()

,combined,precip,vis,temp,wind
0,ATL2018-01-01 00:00:00,0.0,10.0,26.1,17.0
1,ATL2018-01-01 01:00:00,0.0,10.0,25.0,16.0
2,ATL2018-01-01 02:00:00,0.0,10.0,23.0,16.0
3,ATL2018-01-01 03:00:00,0.0,10.0,21.9,15.0
4,ATL2018-01-01 04:00:00,0.0,10.0,21.0,17.0


In [49]:
#suming up the precipitation for last five hours
sum_precip = []
for row in range(len(weather_df_new1.precip) - 5):
    rows_sum = sum(weather_df_new1.iloc[row:row + 5, 1].values)
    sum_precip.append(rows_sum)

In [50]:
sum_precip.insert(0,0)
sum_precip.insert(0,0)
sum_precip.insert(0,0)
sum_precip.insert(0,0)
sum_precip.insert(0,0)

In [51]:
#Averaging the visability for last five hours
ave_vis = []
for row in range(len(weather_df_new1.vis) - 5):
    rows_ave = np.mean(weather_df_new1.iloc[row:row + 5, 1].values)
    ave_vis.append(rows_ave)

In [52]:
ave_vis.insert(0,0)
ave_vis.insert(0,0)
ave_vis.insert(0,0)
ave_vis.insert(0,0)
ave_vis.insert(0,0)

In [53]:
weather_df_new1['ave_vis'] = ave_vis

In [54]:
weather_df_new1['precip_sum'] = sum_precip

In [55]:
weather_df_new1 = weather_df_new1.drop(columns=['precip','vis'])

In [56]:
weather_df_new1.head()

,combined,temp,wind,ave_vis,precip_sum
0,ATL2018-01-01 00:00:00,26.1,17.0,0.0,0.0
1,ATL2018-01-01 01:00:00,25.0,16.0,0.0,0.0
2,ATL2018-01-01 02:00:00,23.0,16.0,0.0,0.0
3,ATL2018-01-01 03:00:00,21.9,15.0,0.0,0.0
4,ATL2018-01-01 04:00:00,21.0,17.0,0.0,0.0


In [57]:
#do the same in the flight data 
df['combined'] =  df['Origin'] + df['FL_TIME_NEW'].astype(str)

In [58]:
df.head()

,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,Origin,OriginState,DestState,Dest,CRSDepTime,CRSArrTime,CRSElapsedTime,ArrDelay,Distance,DistanceGroup,FL_TIME,FL_TIME_NEW,combined
0,4,10,14,7,2018-10-14,AA,N925UY,1674,LAX,CA,NC,CLT,1123,1910,287.0,-16.0,2125.0,9,2018-10-14 11:23:00,2018-10-14 11:00:00,LAX2018-10-14 11:00:00
1,4,10,15,1,2018-10-15,AA,N583UW,1674,LAX,CA,NC,CLT,1123,1910,287.0,-21.0,2125.0,9,2018-10-15 11:23:00,2018-10-15 11:00:00,LAX2018-10-15 11:00:00
2,4,10,16,2,2018-10-16,AA,N581UW,1674,LAX,CA,NC,CLT,1123,1910,287.0,-14.0,2125.0,9,2018-10-16 11:23:00,2018-10-16 11:00:00,LAX2018-10-16 11:00:00
3,4,10,17,3,2018-10-17,AA,N551UW,1674,LAX,CA,NC,CLT,1123,1910,287.0,-8.0,2125.0,9,2018-10-17 11:23:00,2018-10-17 11:00:00,LAX2018-10-17 11:00:00
4,4,10,18,4,2018-10-18,AA,N181UW,1674,LAX,CA,NC,CLT,1123,1910,287.0,-4.0,2125.0,9,2018-10-18 11:23:00,2018-10-18 11:00:00,LAX2018-10-18 11:00:00


In [59]:
copy_df = df.copy()

In [60]:
copy_df.head()

,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,Origin,OriginState,DestState,Dest,CRSDepTime,CRSArrTime,CRSElapsedTime,ArrDelay,Distance,DistanceGroup,FL_TIME,FL_TIME_NEW,combined
0,4,10,14,7,2018-10-14,AA,N925UY,1674,LAX,CA,NC,CLT,1123,1910,287.0,-16.0,2125.0,9,2018-10-14 11:23:00,2018-10-14 11:00:00,LAX2018-10-14 11:00:00
1,4,10,15,1,2018-10-15,AA,N583UW,1674,LAX,CA,NC,CLT,1123,1910,287.0,-21.0,2125.0,9,2018-10-15 11:23:00,2018-10-15 11:00:00,LAX2018-10-15 11:00:00
2,4,10,16,2,2018-10-16,AA,N581UW,1674,LAX,CA,NC,CLT,1123,1910,287.0,-14.0,2125.0,9,2018-10-16 11:23:00,2018-10-16 11:00:00,LAX2018-10-16 11:00:00
3,4,10,17,3,2018-10-17,AA,N551UW,1674,LAX,CA,NC,CLT,1123,1910,287.0,-8.0,2125.0,9,2018-10-17 11:23:00,2018-10-17 11:00:00,LAX2018-10-17 11:00:00
4,4,10,18,4,2018-10-18,AA,N181UW,1674,LAX,CA,NC,CLT,1123,1910,287.0,-4.0,2125.0,9,2018-10-18 11:23:00,2018-10-18 11:00:00,LAX2018-10-18 11:00:00


In [61]:
weather_df_new1.head()

,combined,temp,wind,ave_vis,precip_sum
0,ATL2018-01-01 00:00:00,26.1,17.0,0.0,0.0
1,ATL2018-01-01 01:00:00,25.0,16.0,0.0,0.0
2,ATL2018-01-01 02:00:00,23.0,16.0,0.0,0.0
3,ATL2018-01-01 03:00:00,21.9,15.0,0.0,0.0
4,ATL2018-01-01 04:00:00,21.0,17.0,0.0,0.0


In [62]:
#mapping the weather data to the flight dataframe

In [63]:
weather_df_new1['zip'] = list(zip(weather_df_new1.temp, weather_df_new1.wind, weather_df_new1.ave_vis,
                                  weather_df_new1.precip_sum))

In [64]:
weather_df_new1.head()

,combined,temp,wind,ave_vis,precip_sum,zip
0,ATL2018-01-01 00:00:00,26.1,17.0,0.0,0.0,"(26.1, 17.0, 0.0, 0.0)"
1,ATL2018-01-01 01:00:00,25.0,16.0,0.0,0.0,"(25.0, 16.0, 0.0, 0.0)"
2,ATL2018-01-01 02:00:00,23.0,16.0,0.0,0.0,"(23.0, 16.0, 0.0, 0.0)"
3,ATL2018-01-01 03:00:00,21.9,15.0,0.0,0.0,"(21.9, 15.0, 0.0, 0.0)"
4,ATL2018-01-01 04:00:00,21.0,17.0,0.0,0.0,"(21.0, 17.0, 0.0, 0.0)"


In [65]:
look_up = weather_df_new1.copy().drop(columns=['temp','wind','ave_vis','precip_sum'])

In [66]:
look_up = look_up.set_index('combined')

In [67]:
look_up.head()

,zip
combined,
ATL2018-01-01 00:00:00,"(26.1, 17.0, 0.0, 0.0)"
ATL2018-01-01 01:00:00,"(25.0, 16.0, 0.0, 0.0)"
ATL2018-01-01 02:00:00,"(23.0, 16.0, 0.0, 0.0)"
ATL2018-01-01 03:00:00,"(21.9, 15.0, 0.0, 0.0)"
ATL2018-01-01 04:00:00,"(21.0, 17.0, 0.0, 0.0)"


In [68]:
weather_dict = look_up.to_dict()['zip']

In [69]:
copy_df['zipped'] = copy_df.combined.map(weather_dict)

In [71]:
copy_df = copy_df.fillna(0)

In [73]:
copy_df['zipped'] = copy_df.zipped.apply(lambda x: (0,0,0,0) if x==0 else x)

In [76]:
 copy_df[['tempF','wind','ave_vis','precip_sum']] = pd.DataFrame(copy_df['zipped'].tolist(), index=df.index)                                                                                                                       

In [78]:
#export the dataframe as a csv to model
copy_df.to_csv('./Data/2018NEW.csv', index = False, encoding = 'utf-8')